In [ ]:
import json
import pymongo
import operator

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

  Creates database 'Data Tracker' and collection 'products'.

In [ ]:
mydb = myclient["Data_Tracker"]
product_col = mydb["product_col"]

  This function calls the requested collection and print it out.

In [ ]:
def display(collection):
    try:
        name = mydb[collection]
        for i in name.find():
            print(i)
    except:
        pass

  Imports data from a JSON file received from SQL database.

In [ ]:
with open("data.json", "r") as line:
    data = json.load(line)
    for i in data: product_col.update(i, i, upsert=True)

In [ ]:
print(f"[*] Database:\n\n")
display("product_col")

  Creates collection for top three products.

In [ ]:
top_products_col = mydb["top_three_products"]
#---------------------------------------------------------------------------------

  The var 'top_items' selects and returns the first objects from the database.<br>
  In line 38 we add the three objects to the collection top_three_products

In [ ]:
top_items = product_col.find().limit(3)

In [ ]:
try:
    for i in top_items: top_products_col.update(i, i, upsert=True)
except:
    pass

In [ ]:
print(f"\n\n[*] Top Three Products:\n\n")
for i in product_col.find().limit(3): print(i)
#---------------------------------------------------------------------------------

  Sorts all the data by their product type in descending order.

In [ ]:
product_col.find().sort("Type", -1)
print(f"\n\n[*] Descending Data: \n\n")
display("product_col")
#---------------------------------------------------------------------------------

  Deletes two products from my top three products.

In [ ]:
top_products_col.delete_many({"Type":"cooldrink"})
print(f"\n\n[*] Deleted Two Objs:\n\n")
display('top_three_products')
#---------------------------------------------------------------------------------

  Updates 1 product and its brands.

In [ ]:
item_brand = {"Type": "chips", "Brand/product": "Lays"}
new_brand = {"id": 1, "Type": "cooldrink","Brand/product": "Coke Zero", "Price per unit":7.5, "Stock":124, "Total stock price":930}

In [ ]:
mydb.product_col.update(item_brand, new_brand, upsert=True)
print("\n\n[*] Updated Product:\n\n")
for i in product_col.find({"Brand/product": "Sprite"}): print(i) 
#---------------------------------------------------------------------------------

  Search for five brands.

In [ ]:
brand_list = ['Simba', 'Monster', 'BarOne', 'Doritos', 'Fanta']

In [ ]:
print("\n\n[*] Items Searched\n\n")

In [ ]:
item_types = ["chips", "cooldrink", "chocolate", "pies", "cupcakes", "veggies", "fruit", "energy_drink", "sauce"]
items = {}

  This loop gathers all objects/dictionaries that contain the item names that are housed within item_types.<br>
  This loop then adds values to the dictionary items it add the key value and how many objects there are of<br>
  that kind. Example: {'chips':4}

In [ ]:
for i in item_types:
    a = product_col.find({'Type': {'$regex': f'{i}.*'}})
    items[i] = a.count()

  This sorts the dictionaries values in numerical order.

In [ ]:
data = sorted(items.items(), key=operator.itemgetter(1))

  Returns the lest 5 types along with all their objects.

In [ ]:
i = 0
while i < 5:
   b = data[i][0]
   c = product_col.find({"Type":b})
   for item in c: print(item)
   i+= 1
#---------------------------------------------------------------------------------

In [ ]:
myclient.close()